In [ ]:
!pip install keras-tuner &> /dev/null
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch,BayesianOptimization
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,f1_score,roc_auc_score,precision_score, recall_score
import pickle

In [ ]:
#train test split
X_train_brty, X_test_brty, y_train_brty, y_test_brty = train_test_split(X_brty, y_brty, test_size = 0.3,stratify=y_brty)

#-----------------------------------------------------------------------------------------------
#--------------------------------HYPER-TUNE MODEL-----------------------------------------------
#-----------------------------------------------------------------------------------------------
#same like RandomSearch in Machine learning, we need to setup different combination of hyperparameter. I am using Keras Tuner API for this

def model_tune(hyperparameter,**kwargs):
  model = keras.Sequential() #initialize seq model
  #tune epoch and batch_size so i am giving a range
  kwargs['batch_size'] = hyperparameter.Int('batch_size', 8, 64, step=4)
  kwargs['epochs'] = hyperparameter.Int('epochs', 100, 500, step=5)
  drop_outs = hyperparameter.Float('drop_outs',0, 0.5, step=0.1, default=0)

  model.add(tf.keras.layers.InputLayer(input_shape=702)) #I donot want change anything on my input layer so it is outside the tune loop
  # Tune the number of dense layers 
  for i in range(hyperparameter.Int('num_layers', 1, 5)):
    hyperparameter_nodes = hyperparameter.Int('units_'+str(i), min_value=8, max_value=128, step=8) #for every layer I used number of nodes ranging from 8 - 1024
    model.add(tf.keras.layers.Dense(units=hyperparameter_nodes, activation='relu'))
    model.add(Dropout(drop_outs))

  model.add(tf.keras.layers.Dense(1,activation='sigmoid')) #this is output layer 

  hyperparameter_learning_rate = hyperparameter.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log") # lets see what learning rate is the best for optimizer
  
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hyperparameter_learning_rate),loss='binary_crossentropy',metrics=[keras.metrics.BinaryAccuracy()]) 

  return model

tuned = RandomSearch(model_tune, objective='val_binary_accuracy',max_trials=50) # I am trying 300 different combination of neural networks
tuned.search(X_train_brty, y_train_brty, verbose = 1,validation_split = 0.2) #let call randomsearch and run hypertuning

epochs = (tuned.get_best_hyperparameters(num_trials=1)[0]).get("epochs")
batch_size = (tuned.get_best_hyperparameters(num_trials=1)[0]).get("batch_size")

tuned_model = tuned.hypermodel.build(tuned.get_best_hyperparameters(num_trials=1)[0])

#we do not want to keep on training if the model is not doing good. so early stopping will stop model from training further
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy')
history = tuned_model.fit(X_train_brty, y_train_brty, verbose = 1,validation_split = 0.2,epochs = epochs,batch_size = batch_size)
pred_test = tuned_model.predict(X_test_brty)
y_pred_brty = np.where(pred_test > 0.5, 1, 0)